# One Device Strategy

In this notebook, we'll explore how to set up a [One Device Strategy](https://www.tensorflow.org/api_docs/python/tf/distribute/OneDeviceStrategy). This strategy is especially useful for deliberately testing code on a single device. Employing this strategy is a prudent first step before advancing to more complex distribution strategies that operate across multiple devices. This approach helps in identifying issues early in the development process, ensuring that our code functions correctly in a simpler environment before transitioning to distributed computing configurations.

## Imports

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

## Define the Distribution Strategy

We can list the available devices on our machine and specify a particular device type to use. This step is crucial for verifying the exact device name that we need to pass into `tf.distribute.OneDeviceStrategy()`. By explicitly identifying and specifying the device, we ensure that TensorFlow directs its computational tasks to the correct hardware, whether it's a specific GPU or CPU. This precise control is essential for optimizing performance and for targeted testing on a single device before scaling up to more complex multi-device strategies.

In [ ]:
# Choose a device type such as CPU or GPU
devices = tf.config.list_physical_devices('GPU')
print(devices[0])

# We'll see that the name will look something like "/physical_device:GPU:0"
# Just take the GPU:0 part and use that as the name
gpu_name = "GPU:0"

# Define the strategy and pass in the device name
one_strategy = tf.distribute.OneDeviceStrategy(device=gpu_name)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Parameters

We'll set up a few global variables to establish the foundational parameters for our model and dataset. These variables will help streamline the configuration process, ensuring that our setup remains organized and consistent throughout the training and evaluation phases. This approach not only enhances code readability but also simplifies adjustments and scalability in future development stages.

In [ ]:
pixels = 224
MODULE_HANDLE = 'https://tfhub.dev/tensorflow/resnet_50/feature_vector/1'
IMAGE_SIZE = (pixels, pixels)
BATCH_SIZE = 32

print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

Using https://tfhub.dev/tensorflow/resnet_50/feature_vector/1 with input size (224, 224)


## Download and Prepare the Dataset

We will use the [Cats vs Dogs](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs) dataset, which we will fetch using TensorFlow Datasets (TFDS). This dataset is a popular choice for practicing image classification techniques, providing a balanced set of images of cats and dogs. Using TFDS for fetching the dataset ensures that we can easily access and preprocess the data, making it ready for training our model efficiently.

In [ ]:
splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']

(train_examples, validation_examples, test_examples), info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete9O56S2/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [ ]:
# Resize the image and normalize pixel values
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [ ]:
# Prepare batches
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [ ]:
# Check if the batches have the correct size and the images have the correct shape
for image_batch, label_batch in train_batches.take(1):
    pass

print(image_batch.shape)

(32, 224, 224, 3)


## Define and Configure the Model

As with other TensorFlow distribution strategies, setting up our model with the One Device Strategy requires minimal code changes. To streamline the process, let's first define a utility function that will both build and compile our model. This function will encapsulate all necessary steps for model construction, including defining the architecture and setting up the compilation parameters like the optimizer, loss function, and metrics. This approach ensures that our model setup is not only efficient but also easily adaptable to different scenarios or distribution strategies.

In [ ]:
# Freeze the layer weights of our feature extractor during training
do_fine_tuning = False

In [ ]:
def build_and_compile_model():
    print("Building model with", MODULE_HANDLE)

    # Configure the feature extractor fetched from TF Hub
    feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,),
                                   trainable=do_fine_tuning)

    # Define the model
    model = tf.keras.Sequential([
      feature_extractor,
      # Append a dense with softmax for the number of classes
      tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    # Display summary
    model.summary()

    # Configure the optimizer, loss and metrics
    optimizer = tf.keras.optimizers.SGD(lr=0.002, momentum=0.9) if do_fine_tuning else 'adam'
    model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

We can now call our model-building function within the strategy scope. This practice places all variables and computations on the device we specified earlier. By invoking the function in this context, TensorFlow ensures that all model components are appropriately allocated to the designated device, optimizing computational efficiency and resource utilization. This step is crucial for leveraging the full capabilities of the selected hardware during model training and operations.


In [ ]:
# Build and compile under the strategy scope
with one_strategy.scope():
    model = build_and_compile_model()

Building model with https://tfhub.dev/tensorflow/resnet_50/feature_vector/1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              23561152  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,565,250
Trainable params: 4,098
Non-trainable params: 23,561,152
_________________________________________________________________


In [ ]:
EPOCHS = 5

# `model.fit()` can be used as we usually do
hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=validation_batches)

Epoch 1/5
582/582 [==============================] - 106s 115ms/step - loss: 0.0364 - accuracy: 0.9874 - val_loss: 0.0346 - val_accuracy: 0.9897
Epoch 2/5
582/582 [==============================] - 72s 117ms/step - loss: 0.0193 - accuracy: 0.9935 - val_loss: 0.0268 - val_accuracy: 0.9918
Epoch 3/5
582/582 [==============================] - 72s 117ms/step - loss: 0.0144 - accuracy: 0.9953 - val_loss: 0.0259 - val_accuracy: 0.9923
Epoch 4/5
582/582 [==============================] - 72s 116ms/step - loss: 0.0128 - accuracy: 0.9955 - val_loss: 0.0314 - val_accuracy: 0.9910
Epoch 5/5
582/582 [==============================] - 71s 116ms/step - loss: 0.0094 - accuracy: 0.9965 - val_loss: 0.0320 - val_accuracy: 0.9918


Once we confirm that everything is functioning correctly on a single device using the One Device Strategy, we can confidently transition to a different device or shift to a more complex distribution strategy that spans multiple devices. This progressive approach ensures that the foundational elements of the model and data handling are robust before scaling up, minimizing potential issues in more complex distributed environments. This step is key to leveraging distributed computing to enhance performance and efficiency as model demands increase.